In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


fname = 'Elec_1'
log_file = fname + '.txt'

with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')


df = pd.read_csv('datasets/ElecMorocco2016.csv', encoding='utf-8', engine='python', sep='\t') #, quotechar="'"  , quoting=3


      
display(df.columns)
display(len(df))
display(df[:4])



classes = set(df['sentiment'].values)
display(classes)

c = df['sentiment'].value_counts()
display(c)

df['sentiment'] = df['sentiment'].astype('category')
df['label'] = df['sentiment'].cat.codes

df = df[['comment_message', 'label']]
classes_num = len(classes)
display(classes_num)
display(len(df))


# ds = Dataset.from_pandas(df)
# ds = ds.train_test_split(test_size=0.2)

# display(ds)

max_sequence_length = 128


models = [ 
        'aubmindlab/bert-base-arabertv02-twitter',
        'CAMeL-Lab/bert-base-arabic-camelbert-da',
        'qarib/bert-base-qarib', 
        'UBC-NLP/MARBERT',
        'UBC-NLP/MARBERTv2',
    'SI2M-Lab/DarijaBERT-mix',
    'SI2M-Lab/DarijaBERT-arabizi'
]


seeds = [0, 1, 42]



for model_name in models:
    for seed in seeds:
        ds = Dataset.from_pandas(df)
        ds = ds.train_test_split(test_size=0.2, seed = seed)
        if seed==0:
            display(ds)
        for i in range(3):
            print(f'{model_name}, try:{i}')
                  
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                                  num_labels=classes_num).to('cuda')                                                 
            dataset_train = ds['train']
            dataset_validation = ds['test']                                                    
            
          
    
            def preprocess_function(examples):
                return tokenizer(examples['comment_message'], truncation=True, padding="max_length",
                                max_length=max_sequence_length)
            
            
            dataset_train = dataset_train.map(preprocess_function, batched=True) # , batched=True
            dataset_validation = dataset_validation.map(preprocess_function, batched=True)  # , batched=True
            
           
            
            def compute_metrics(eval_pred):
                logits, labels = eval_pred
                predictions = np.argmax(logits, axis=-1)    
                acc = accuracy_score(labels, predictions)        
                f1 = f1_score(labels, predictions, average='macro')   
                with open(log_file, 'a') as f:
                    f.write(f'{model_name},{acc},{f1}\n')
                return {'accuracy': acc, 'f1_score': f1}
    
    
            
            
            epochs = 15
            save_steps = 10000 #save checkpoint every 10000 steps
            batch_size = 64
            
            training_args = TrainingArguments(
                output_dir = 'bert/',
                overwrite_output_dir=True,
                num_train_epochs = epochs,
                per_device_train_batch_size = batch_size,
                per_device_eval_batch_size = batch_size,
                save_steps = save_steps,
                save_total_limit = 1, #only save the last 5 checkpoints
                fp16=True,
                learning_rate = 5e-5,  # 5e-5 is the default
                logging_steps = 60, #50_000
                evaluation_strategy = 'steps',
                # evaluate_during_training = True,
                eval_steps = 60
                
            )
            
            trainer = Trainer(
                model = model,
                args = training_args,
                # data_collator=data_collator,
                train_dataset=dataset_train,
                eval_dataset=dataset_validation,
                compute_metrics = compute_metrics
            )
            
            
            trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv(f'{fname}.csv')
display(best_results)



2024-09-17 08:39:20.189277: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-17 08:39:20.214738: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-17 08:39:20.636209: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Index(['index', 'comment_message', 'sentiment', 'comment_published'], dtype='object')

10254

,index,comment_message,sentiment,comment_published
0,1,البكاء ليس سمت الرجال ..البكاء سلطة الضعفاء لكسب التعاطف ..انا لست ضد بنكيران وانما الرجل الصالح هو الذي يفعل ما يقول ولا يبكي الا عند النصر,N,2016-10-01 00:07:44
1,2,كلنا بنكيران\nولاية ثانية ان شاء الله\nموتوا بحقدكم,P,2016-10-01 00:47:37
2,3,الشركات في الدول المتقدمة تساهم في بناء أوطانها بدفع الضرائب وفي بلدنا العزيز الشركات تطلب من الدولة أن تخدم عليها؟,N,2016-10-01 00:48:02
3,4,من ينتقد بن كران هم خصوم معارضة لى خاصها هي سلطة لا تهتم لوضعية مواطن,N,2016-10-01 00:49:10


{'N', 'P'}

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at SI2M-Lab/DarijaBERT-arabizi and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/8203 [00:00<?, ? examples/s]

Map:   0%|          | 0/2051 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
60,0.505100,0.445808,0.798635,0.780468
120,0.452000,0.439255,0.810336,0.768018
180,0.333400,0.450055,0.818137,0.786077
240,0.323200,0.446000,0.810824,0.792268
300,0.233900,0.499006,0.824963,0.795896
360,0.181300,0.543101,0.807899,0.789723
420,0.122600,1.119851,0.759629,0.751270
480,0.092800,0.806001,0.800585,0.788092
540,0.072800,0.962623,0.809361,0.792163
600,0.048800,1.095183,0.784495,0.773849


,Model,Accuracy,F1
0,CAMeL-Lab/bert-base-arabic-camelbert-da,0.852755,0.836816
3,SI2M-Lab/DarijaBERT-arabizi,0.835690,0.819515
6,SI2M-Lab/DarijaBERT-mix,0.861043,0.846054
9,UBC-NLP/MARBERT,0.874695,0.858599
12,UBC-NLP/MARBERTv2,0.884447,0.871189
15,aubmindlab/bert-base-arabertv02-twitter,0.883471,0.870011
18,qarib/bert-base-qarib,0.871770,0.858597
